<a href="https://colab.research.google.com/github/danielmlow/tutorials/blob/main/text/openrouter_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def openrouter_request(prompt, OPENROUTER_API_KEY, model = 'meta-llama/llama-3.1-70b-instruct:free',
                       temperature = 0):
  """
  free models: 20 requests per minute and 200 requests per day. See https://openrouter.ai/docs/limits
  """



  response = requests.post(
      url="https://openrouter.ai/api/v1/chat/completions",
      headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",

      },
      data=json.dumps({
        "model": model, # Optional
        "temperature": temperature,
        "response_format": { "type": "json_object" },
        "messages": [
          { "role": "user", "content": prompt}
        ]

      })
      )
  data = json.loads(response.text.strip())
  try:
    final_result = data['choices'][0]['message']['content']
    final_result = dict(eval(final_result))
    return final_result, data
  # raise error
  except KeyError as e:
        if str(e) == "choices":
            print(data)
            raise
        else:
            raise  # re-raise any other KeyError


In [ ]:



prompt_template = """Classify the text into one or more of the following {context} symptoms: {symptoms}.

Provide a certainty score (between 0 and 1) and an explanation for each chosen symptom (words or phrases from the text that are related to the symptom).

Text: {text}

Structure your response in the following JSON format:
{{'symptom_A': [[certainty_score], [words, phrases]], 'symptom_B': [[certainty_score], [words, phrases]], ...}}

JSON:
"""

In [ ]:

symptoms =

In [ ]:

prompt = prompt_template.format(context = 'mental health',
              symptoms = symptoms,
              text= 'No one cares about me. I go to therapy, but it doesnt work. It wont get better. I want out. Im feeling hyperactive.'
              )


print(prompt)

In [ ]:
OPENROUTER_API_KEY = "" # get it from OpenRouter AI
model  = 'meta-llama/llama-3.1-70b-instruct:free' # or "gpt-4o-mini" or some other one

In [ ]:
final_result, data = openrouter_request(prompt, OPENROUTER_API_KEY, model = model, temperature=0)

In [ ]:
#then you